## Background 
 -- *From Romero and Rosokha ([2018](https://www.sciencedirect.com/science/article/pii/S0014292118300394),[2019a](https://www.aeaweb.org/articles?id=10.1257/mic.20160220),[2019b](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3290732)) and Rosokha and Wei ([2020](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3526505))*

Strategy frequency estimation method (SFEM) is a finite-mixture estimation aproach to estimate the proportion of strategies in experimental data proposed by Dal Bo and Frechette (2011). The method works by first specifying the set of $K$ strategies considered by the modeler. Then, for each subject $n\in \{1,...,N\}$, and each strategy $k\in \{1,...,K\}$, the method prescribes to compare subject $n$'s actual play with how strategy $k$ would have played in her place. Let $C(k,n)$ denote the number of periods in which subject $n$'s play correctly matches the play of strategy $k$. Then, let $C$ denote a $K \times N$ matrix of the number of correct matches for all combinations of subjects and strategies. Similarly, let $E$ denote a $K \times N$ matrix of the number of mismatches when comparing subjects' play with what the strategies would do in their place. Then, define a Hadamard-product $P$:

\begin{equation}
    P = \beta^{C}\circ(1-\beta)^{E},
\end{equation}

where $\beta$ is the probability that a subject plays according to a strategy and $(1-\beta)$ the probability that the subjects deviates from that strategy. Thus,
each entry $P(k,n)$ is the likelihood that the observed choices by subject $n$ were generated by strategy $k$. Then, using the matrix dot product, we define the log-likelihood function $\mathcal{L}$:

\begin{equation}
    \mathcal{L}(\beta,\phi)= ln \big( \phi' \cdot P \big) \cdot  \mathbf{1} .
\end{equation}

For this example, the set of strategies is the 20 strategies commonly used in the literature on repeated Prisoner's Dilemma (Fudenberg, Rand, and Dreber 2012). For details, see Appendix H of Romero and Rosokha (2018).

## Step 0: Load libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

## Step 1: Get Data

In [2]:
#For this example we will use data from Romero & Rosokha (2018)
data=pd.read_csv('input\\action_data_RR2018.csv')
data.head()

,subject,supergame,period,action,opponentAction
0,T1S1S01,1,1,C,D
1,T1S1S01,1,2,D,D
2,T1S1S01,1,3,C,D
3,T1S1S01,1,4,D,D
4,T1S1S01,1,5,C,D


## Step 2: Clean Data

In [3]:
# Subset data to supergames and periods of interest. 
# In RR2018 we focus on supermages 31-50 and restrict to at most 20 first periods 
# This is done so numerical issues when raising beta to large powers
data = data[(data.supergame>30) & (data.supergame<51) & (data.period<21)] 

In [4]:
# Get number of subjects and maximum number of actions for each subject 
n_subjects = len(data.subject.unique())
n_actions = data.groupby('subject').count().action.max()

In [5]:
#Create nan matrices of actions, matches, periods, and opponent's actions
actions=np.empty((n_subjects,n_actions))
actions.fill(np.nan)
matches=np.empty((n_subjects,n_actions))
matches.fill(np.nan)
periods=np.empty((n_subjects,n_actions))
periods.fill(np.nan)
others=np.empty((n_subjects,n_actions))
others.fill(np.nan)

In [6]:
# Get the data into matrix format 
for i,sub in enumerate(data.subject.unique()):
    
    a = [x=='C' for x in data.action[data.subject==sub].tolist()]
    m = data.supergame[data.subject==sub].tolist()
    p = data.period[data.subject==sub].tolist()
    o = [x=='C' for x in data.opponentAction[data.subject==sub].tolist()]
    
    n = len(a)
    
    actions[i,:n] = a
    matches[i,:n] = m
    periods[i,:n] = p
    others[i,:n]  = o    

## Step 3: Simulate Strategies Against Opponent Actions

In [7]:
import sys
sys.path.append('input\\')
import strategies

In [8]:
strats = []
strat_names = []
for i in dir(strategies):
    s = getattr(strategies,i)
    if callable(s):
        strats.append(s)
        strat_names.append(s.__name__)
        

n_strats = len(strats)
print("There are",n_strats,'strategies in the strategies.py file. \nThe strategies are:',strat_names)

There are 20 strategies in the strategies.py file. 
The strategies are: ['s01_allc', 's02_alld', 's03_tft', 's04_dtft', 's05_tf2t', 's06_tf3t', 's07_2tft', 's08_2tf2t', 's09_t2', 's10_grim', 's11_grim2', 's12_grim3', 's13_wsls', 's14_2wsls', 's15_ctod', 's16_dtf2t', 's17_dtf3t', 's18_dgrim2', 's19_dgrim3', 's20_dcalt']


In [9]:
# For each subject n and each strategy k compare subject n's actual play with how strategy k would have played.
C = np.zeros((n_strats,n_subjects)) #Number of periods in which play matches
E = np.zeros((n_strats,n_subjects)) #Number of periods in which play does not match
for n in range(n_subjects):
    for k in range(n_strats): 

        subChoice = actions[n]
        otherChoice = others[n]
        periodData = periods[n]

        stratChoice = strats[k](otherChoice,periodData)

        C[k,n]=np.sum(subChoice==stratChoice)
        E[k,n]=np.sum((1-subChoice)==stratChoice)

#Output C and E matrices as an intermediate step
np.savetxt("output\\03-cMatrix.csv", C, delimiter=",")
np.savetxt("output\\03-eMatrix.csv", E, delimiter=",")

## Step 4: Set up the loglikelihood function

In [10]:
# Likelhood function takes as an input a vector of proportions of strategies and returns the likelihood value
#Note cMat and eMat are global matrices that are updated externally for each treatment.
def objective(x,args):
    
    C = args[0]
    E = args[1]
    
    bc=np.power(x[0],C) #beta to the power of C
    be=np.power(1-x[0],E) #beta to the power of E
    prodBce = np.multiply(bc,be) #Hadamard product
    
    #maximum is taken so that there is no log(0) warning/error
    res = np.log(np.maximum(np.dot(x[1:],prodBce),np.nextafter(0,1))).sum() 
    
    return -res

def constraint1(x):
    
    return x[1:].sum()-1

#Set up the boundaries and constraints
b0 = (np.nextafter(0.5,1),1-np.nextafter(0,1))
b1 = (np.nextafter(0,1),1-np.nextafter(0,1))
bnds = tuple([b0]+[b1]*n_strats) #Beta is at least .5
con1 = {'type': 'eq', 'fun': constraint1} 
cons = ([con1])

## Step 5: Run likelihood maximization

In [11]:
#Some random starting point
x0 = np.zeros(n_strats+1)
x0[0] = .5+.5*np.random.random()
temp = np.random.random(n_strats)
x0[1:]=temp/temp.sum()

bestX=x0
bestObjective=objective(x0,[C,E])

for k in range(50): #Do many times so that there is low chance of getting stuck in local optimum

    x0 = np.zeros(n_strats+1)
    x0[0] = .5+.5*np.random.random()
    temp = np.random.random(n_strats)
    x0[1:]=temp/temp.sum()

    #Notice that we are minimizing the negative
    solution = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons,args=([C,E]))
    x = solution.x
    obj = solution.fun

    if bestObjective>obj:
        bestObjective=obj
        bestX=x

## Step 6: Output results

In [12]:
results=pd.DataFrame(bestX.round(4).tolist()+[np.round(-bestObjective,4)],index=['beta']+strat_names+['LL'])
results=results.rename(columns={0:'Estimates'})
results=results.sort_values(by=['Estimates'],ascending=False)
print(results)
results.to_csv("output\\06-sfem_results.csv")

            Estimates
beta           0.9600
s03_tft        0.3267
s10_grim       0.1439
s02_alld       0.1220
s04_dtft       0.0976
s07_2tft       0.0780
s05_tf2t       0.0737
s01_allc       0.0488
s13_wsls       0.0245
s16_dtf2t      0.0244
s15_ctod       0.0218
s11_grim2      0.0179
s09_t2         0.0126
s06_tf3t       0.0084
s12_grim3      0.0000
s14_2wsls      0.0000
s08_2tf2t      0.0000
s17_dtf3t      0.0000
s18_dgrim2     0.0000
s19_dgrim3     0.0000
s20_dcalt      0.0000
LL         -3871.8297


## References

- Dal Bó, P. and Fréchette, G.R., 2011. [The evolution of cooperation in infinitely repeated games: Experimental evidence.](https://www.aeaweb.org/articles?id=10.1257/aer.101.1.411) American Economic Review, 101(1), pp.411-29.

- Dal Bó, P. and Fréchette, G.R., 2018. [On the determinants of cooperation in infinitely repeated games: A survey.](https://www.aeaweb.org/articles?id=10.1257/jel.20160980) Journal of Economic Literature, 56(1), pp.60-114.

- Fudenberg, D., Rand, D.G. and Dreber, A., 2012. [Slow to anger and fast to forgive: Cooperation in an uncertain world.](https://www.aeaweb.org/articles?id=10.1257/aer.102.2.720) American Economic Review, 102(2), pp.720-49.

- Romero, J. and Rosokha, Y., 2018. [Constructing strategies in the indefinitely repeated prisoner’s dilemma game.](https://www.sciencedirect.com/science/article/pii/S0014292118300394) European Economic Review, 104, pp.185-219.

- Romero, J. and Rosokha, Y., 2019. [The Evolution of Cooperation: The Role of Costly Strategy Adjustments.](https://www.aeaweb.org/articles?id=10.1257/mic.20160220) American Economic Journal: Microeconomics, 11(1), pp.299-328.

- Romero, J. and Rosokha, Y., 2019. [Mixed Strategies in the Indefinitely Repeated Prisoner's Dilemma.](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3290732) Available at SSRN 3290732.

- Rosokha, Y. and Wei, C., 2020. [Cooperation in Queueing Systems.](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3526505) Available at SSRN 3526505.
